<a href="https://colab.research.google.com/github/TimH2024/MSC-M5-Project/blob/main/5_Std_dataset_Hyper_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Hyper Parameter Tuning Results

# INSTALL LIBRARIES

In [ ]:
pip install tensorflow keras keras-tuner numpy pandas scikit-learn

In [ ]:
# Core libraries
import numpy as np
import pandas as pd

# Machine learning and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# TensorFlow/Keras for deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Keras Tuner for hyperparameter tuning
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

# LOAD FILES

In [ ]:
import os
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Define the full file path
master_results_file_path = os.path.join(subdirectory, 'master_resultsStd.csv')

# Check if the file exists and load it
if os.path.exists(master_results_file_path):
    master_resultsStd = pd.read_csv(master_results_file_path)
    print(f"File 'master_resultsStd.csv' loaded successfully!")
    print(master_resultsStd.head())  # Display the first few rows of the DataFrame
else:
    print(f"File 'master_resultsStd.csv' not found in '{subdirectory}'. Please check the file path.")

Mounted at /content/drive
File 'master_resultsStd.csv' loaded successfully!
      Target                        Model            Train R²   Test R²  \
0  new_price            Linear Regression  0.6297378908089446  0.627819   
1  new_price      Random Forest Regressor   0.973289789332776  0.824091   
2  new_price            XGBoost Regressor  0.8270074320236708  0.766067   
3  new_price  Gradient Boosting Regressor  0.7699284396872543  0.746053   
4         PC            Linear Regression   0.973660068974925  0.974082   

        MAE       MSE      RMSE        MAPE Comments                      Type  
0  0.489318  0.373920  0.611490  249.308212   Normal  Simple Linear Regression  
1  0.282348  0.176731  0.420393  138.904649   Normal       Decision Tree Model  
2  0.365720  0.235026  0.484795  173.392079   Normal       Decision Tree Model  
3  0.383921  0.255134  0.505108  194.961371   Normal       Decision Tree Model  
4  0.131777  0.025915  0.160980   29.441769   Normal  Simple Linear 

In [ ]:
master_resultsStd.head(20)

,Target,Model,Train R²,Test R²,MAE,MSE,RMSE,MAPE,Comments,Type
0,new_price,Linear Regression,0.6297378908089446,0.627819,4.893183e-01,3.739195e-01,6.114896e-01,2.493082e+02,Normal,Simple Linear Regression
1,new_price,Random Forest Regressor,0.973289789332776,0.824091,2.823479e-01,1.767307e-01,4.203935e-01,1.389046e+02,Normal,Decision Tree Model
2,new_price,XGBoost Regressor,0.8270074320236708,0.766067,3.657201e-01,2.350260e-01,4.847948e-01,1.733921e+02,Normal,Decision Tree Model
3,new_price,Gradient Boosting Regressor,0.7699284396872543,0.746053,3.839207e-01,2.551336e-01,5.051075e-01,1.949614e+02,Normal,Decision Tree Model
4,PC,Linear Regression,0.973660068974925,0.974082,1.317771e-01,2.591456e-02,1.609800e-01,2.944177e+01,Normal,Simple Linear Regression
5,PC,Random Forest Regressor,1.0,1.000000,1.665155e-14,4.991350e-28,2.234133e-14,1.707175e-12,Normal,Decision Tree Model
6,PC,XGBoost Regressor,0.9999566580170176,0.999957,6.098938e-03,4.334612e-05,6.583777e-03,6.560287e-01,Normal,Decision Tree Model
7,PC,Gradient Boosting Regressor,0.9999999999999998,1.000000,1.221103e-08,2.160188e-16,1.469758e-08,3.735057e-06,Normal,Decision Tree Model
8,new_price,Base LSTM,0.995441277667468,0.995389,1.088874e+05,2.124968e+10,1.457727e+05,1.696912e+00,Working Well,Neural Network
9,PC,Base LSTM,0.9986385895087232,0.998622,6.581006e-01,6.840771e-01,8.270895e-01,1.799154e-01,Working Well,Neural Network


In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the file path in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/Std_Datasetv5.csv'

# Read the file into a DataFrame
try:
    merged_dataset = pd.read_csv(file_path)
    print("[INFO] File loaded successfully into 'merged_dataset'.")
    print(merged_dataset.head())  # Display the first 5 rows
except FileNotFoundError:
    print(f"[ERROR] File not found at '{file_path}'. Please check the file path and try again.")
except Exception as e:
    print(f"[ERROR] An error occurred while reading the file: {e}")

Mounted at /content/drive
[INFO] File loaded successfully into 'merged_dataset'.
   saleable_area(ft^2)  unit_rate  floor  CPI   PC  IR  MW  M3  SD  UR  ...  \
0                  423      15792   22.0    0  351   3  18  17   0   3  ...   
1                  761      15857   12.0    0  351   3  18  17   0   3  ...   
2                  320      13717   12.0    0  351   3  18  17   0   3  ...   
3                  519      16541   29.0    0  351   3  18  17   0   3  ...   
4                  699      14721   24.0    0  351   3  18  17   0   3  ...   

   floor_height_floor_16to18  floor_height_floor_19to20  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   floor_height_floor_21to25  floor_height_floor_26to30  \
0                          1  

In [ ]:
merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72663 entries, 0 to 72662
Data columns (total 71 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   saleable_area(ft^2)                             72663 non-null  int64  
 1   unit_rate                                       72663 non-null  int64  
 2   floor                                           72663 non-null  float64
 3   CPI                                             72663 non-null  int64  
 4   PC                                              72663 non-null  int64  
 5   IR                                              72663 non-null  int64  
 6   MW                                              72663 non-null  int64  
 7   M3                                              72663 non-null  int64  
 8   SD                                              72663 non-null  int64  
 9   UR                                     

# HYPERPARAMETER TUNING

In [ ]:
# Define the feature matrix (X) and target variables (y)

features = merged_dataset.drop(columns=['new_price', 'PC'])
targets_Std = merged_dataset[['new_price', 'PC']]

X = features.copy()
y = targets_Std.copy()

In [28]:
import os
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from keras_tuner import RandomSearch
import numpy as np
import tensorflow as tf

# File paths
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'
master_results_file_path = os.path.join(subdirectory, 'master_resultsStd.csv')

# Load or initialize master results table
if os.path.exists(master_results_file_path):
    master_resultsStd = pd.read_csv(master_results_file_path)
    print(f"[INFO] Loaded existing master results table from '{master_results_file_path}'.")
else:
    master_resultsStd = pd.DataFrame(columns=["Target", "Model", "Type", "Train R²", "Test R²", "MAE", "MSE", "RMSE", "MAPE", "Comments"])
    print(f"[INFO] Initialized a new master results table.")

# Define features and target variables
features = merged_dataset.drop(columns=['new_price', 'PC'])
targets_Std = merged_dataset[['new_price', 'PC']]
X = features.copy()
y = targets_Std.copy()

# Normalize features and targets
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

target_scalers = {col: StandardScaler() for col in targets_Std.columns}
y_scaled = pd.DataFrame({col: target_scalers[col].fit_transform(targets_Std[[col]]).flatten() for col in targets_Std.columns})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Subset the dataset for faster tuning
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
X_train_sample_lstm = X_train_sample.reshape((X_train_sample.shape[0], 1, X_train_sample.shape[1]))

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)

    for i in range(num_layers):
        return_sequences = hp.Choice(f'return_sequences_{i+1}', values=[True, False]) if i < (num_layers - 1) else False
        model.add(LSTM(
            units=hp.Int(f'units_layer_{i+1}', min_value=32, max_value=128, step=32),
            activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
            return_sequences=return_sequences,
            recurrent_dropout=hp.Choice(f'recurrent_dropout_{i+1}', values=[0.0, 0.2, 0.3]),
            input_shape=(X_train_sample_lstm.shape[1], X_train_sample_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=hp.Choice(f'dropout_layer_{i+1}', values=[0.2, 0.3, 0.4])))

    model.add(Dense(1))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
        loss=hp.Choice('loss', values=['mse', 'mae', 'huber']),
        metrics=['mae']
    )
    return model

# Initialize tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Increased for a better hyperparameter search
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='lstm_hyper_tuning'
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Loop through targets (new_price and PC) to dynamically update results
for target_name in ["new_price", "PC"]:
    if target_name not in y_train.columns:
        raise KeyError(f"Target column '{target_name}' not found in y_train. Available columns are: {y_train.columns.tolist()}")

    # Train and evaluate the model for the current target
    y_train_target_sample = y_train_sample[[target_name]].values

    # Loop through different batch sizes and epochs
    for batch_size in [32, 64, 128]:
        for epochs in [10, 20, 30]:
            print(f"[INFO] Tuning for batch_size={batch_size}, epochs={epochs}")

            # Hyperparameter tuning
            tuner.search(
                X_train_sample_lstm,
                y_train_target_sample,
                validation_split=0.1,
                epochs=epochs,
                batch_size=batch_size,
                callbacks=[early_stopping]
            )

            # Build and train the best model
            best_hps = tuner.get_best_hyperparameters(1)[0]
            best_model = tuner.hypermodel.build(best_hps)
            history = best_model.fit(
                X_train_lstm,
                y_train[[target_name]].values,
                validation_data=(X_test_lstm, y_test[[target_name]].values),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=[early_stopping]
            )

            # Evaluate the model
            train_predictions = best_model.predict(X_train_lstm)
            test_predictions = best_model.predict(X_test_lstm)

            scaler_y = target_scalers[target_name]
            train_predictions_rescaled = scaler_y.inverse_transform(train_predictions)
            test_predictions_rescaled = scaler_y.inverse_transform(test_predictions)

            y_train_rescaled = scaler_y.inverse_transform(y_train[[target_name]])
            y_test_rescaled = scaler_y.inverse_transform(y_test[[target_name]])

            train_r2 = r2_score(y_train_rescaled, train_predictions_rescaled)
            test_r2 = r2_score(y_test_rescaled, test_predictions_rescaled)
            mae = mean_absolute_error(y_test_rescaled, test_predictions_rescaled)
            mse = mean_squared_error(y_test_rescaled, test_predictions_rescaled)
            rmse = np.sqrt(mse)
            mape = np.mean(np.abs((y_test_rescaled - test_predictions_rescaled) / y_test_rescaled)) * 100

            # Add results to the master results table
            new_results = {
                "Target": target_name,
                "Model": "Hyper-Tuned LSTM",
                "Type": "Neural Network",
                "Train R²": train_r2,
                "Test R²": test_r2,
                "MAE": mae,
                "MSE": mse,
                "RMSE": rmse,
                "MAPE": mape,
                "Comments": "Working Well" if test_r2 > 0.75 and mape < 10 else "Needs Improvement"
            }

            # Convert new_results to a DataFrame
            new_results_df = pd.DataFrame([new_results])

            # Check if the target and model already exist in the master results table
            existing_row_index = master_resultsStd[
                (master_resultsStd["Target"] == new_results["Target"]) & (master_resultsStd["Model"] == new_results["Model"])
            ].index

            if len(existing_row_index) > 0:
                # Update the existing row by replacing its values
                master_resultsStd.loc[existing_row_index, :] = new_results_df.iloc[0].values
            else:
                # Append the new results
                master_resultsStd = pd.concat([master_resultsStd, new_results_df], ignore_index=True)

# Save updated results
master_resultsStd.to_csv(master_results_file_path, index=False)
print(f"[INFO] Updated master results saved to '{master_results_file_path}'.")

# Display the final results table
print("\nFinal Master Results Table:")
print(master_resultsStd.to_string(index=False))

Trial 10 Complete [00h 00m 01s]

Best val_loss So Far: 0.008529160171747208
Total elapsed time: 00h 36m 38s
Epoch 1/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 26s 11ms/step - loss: 0.2339 - mae: 0.3243 - val_loss: 0.0056 - val_mae: 0.0550
Epoch 2/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - loss: 0.0251 - mae: 0.1178 - val_loss: 0.0055 - val_mae: 0.0548
Epoch 3/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0197 - mae: 0.1039 - val_loss: 0.0035 - val_mae: 0.0453
Epoch 4/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0177 - mae: 0.0979 - val_loss: 0.0055 - val_mae: 0.0600
Epoch 5/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - loss: 0.0167 - mae: 0.0948 - val_loss: 0.0030 - val_mae: 0.0410
Epoch 6/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0154 - mae: 0.0911 - val_loss: 0.0045 - val_mae: 0.0513
Epoch 7/10
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0151 - mae: 0.0904 - val_loss: 0.0025 - val_mae: 0.0383
Epoch 8/10
1817/1817 ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - loss: 0.2452 - mae: 0.3347 - val_loss: 0.0068 - val_mae: 0.0606
Epoch 2/20
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - loss: 0.0248 - mae: 0.1177 - val_loss: 0.0087 - val_mae: 0.0605
Epoch 3/20
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 39s 11ms/step - loss: 0.0194 - mae: 0.1032 - val_loss: 0.0059 - val_mae: 0.0577
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
[INFO] Tuning for batch_size=32, epochs=30
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1817/1817 ━━━━━━━━━━━━━━━━━━━━ 26s 11ms/step - loss: 0.2276 - mae: 0.3223 - val_loss: 0.0069 - val_mae: 0.0591
Epoch 2/30
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 0.0257 - mae: 0.1199 - val_loss: 0.0055 - val_mae: 0.0559
Epoch 3/30
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - loss: 0.0197 - mae: 0.1036 - val_loss: 0.0064 - val_mae: 0.0575
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
[INFO] Tuning for batch_size=64, epochs=10
Epoch 1/10


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


909/909 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - loss: 0.3141 - mae: 0.3900 - val_loss: 0.0130 - val_mae: 0.0882
Epoch 2/30
909/909 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - loss: 0.0328 - mae: 0.1371 - val_loss: 0.0041 - val_mae: 0.0481
Epoch 3/30
909/909 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - loss: 0.0220 - mae: 0.1101 - val_loss: 0.0047 - val_mae: 0.0490
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
[INFO] Tuning for batch_size=128, epochs=10
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


455/455 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - loss: 0.4168 - mae: 0.4676 - val_loss: 0.0189 - val_mae: 0.1025
Epoch 2/10
 96/455 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0483 - mae: 0.1680

KeyboardInterrupt: 

In [23]:
import os
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from keras_tuner import RandomSearch
import numpy as np
import tensorflow as tf

# File paths
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'
master_results_file_path = os.path.join(subdirectory, 'master_resultsStd.csv')

# Load or initialize master results table
if os.path.exists(master_results_file_path):
    master_resultsStd = pd.read_csv(master_results_file_path)
    print(f"[INFO] Loaded existing master results table from '{master_results_file_path}'.")
else:
    master_resultsStd = pd.DataFrame(columns=["Target", "Model", "Type", "Train R²", "Test R²", "MAE", "MSE", "RMSE", "MAPE", "Comments"])
    print(f"[INFO] Initialized a new master results table.")

# Define features and target variables
features = merged_dataset.drop(columns=['new_price', 'PC'])
targets_Std = merged_dataset[['new_price', 'PC']]
X = features.copy()
y = targets_Std.copy()

# Normalize features and targets
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

target_scalers = {col: StandardScaler() for col in targets_Std.columns}
y_scaled = pd.DataFrame({col: target_scalers[col].fit_transform(targets_Std[[col]]).flatten() for col in targets_Std.columns})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Subset the dataset for faster tuning
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
X_train_sample_lstm = X_train_sample.reshape((X_train_sample.shape[0], 1, X_train_sample.shape[1]))

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)

    for i in range(num_layers):
        return_sequences = i < (num_layers - 1)
        model.add(LSTM(
            units=hp.Int(f'units_layer_{i+1}', min_value=32, max_value=128, step=32),
            activation='relu',
            return_sequences=return_sequences,
            input_shape=(X_train_sample_lstm.shape[1], X_train_sample_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=hp.Choice(f'dropout_layer_{i+1}', values=[0.2, 0.3, 0.4])))

    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])
    ), loss='mse')
    return model

# Initialize tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='lstm_hyper_tuning'
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Loop through targets (new_price and PC) to dynamically update results
for target_name in ["new_price", "PC"]:
    if target_name not in y_train.columns:
        raise KeyError(f"Target column '{target_name}' not found in y_train. Available columns are: {y_train.columns.tolist()}")

    # Train and evaluate the model for the current target
    y_train_target_sample = y_train_sample[[target_name]].values

    # Hyperparameter tuning
    tuner.search(
        X_train_sample_lstm,
        y_train_target_sample,
        validation_split=0.1,
        epochs=20,
        batch_size=64,
        callbacks=[early_stopping]
    )

    # Build and train the best model
    best_hps = tuner.get_best_hyperparameters(1)[0]
    best_model = tuner.hypermodel.build(best_hps)
    history = best_model.fit(
        X_train_lstm,
        y_train[[target_name]].values,
        validation_data=(X_test_lstm, y_test[[target_name]].values),
        epochs=20,
        batch_size=64,
        callbacks=[early_stopping]
    )

    # Evaluate the model
    train_predictions = best_model.predict(X_train_lstm)
    test_predictions = best_model.predict(X_test_lstm)

    scaler_y = target_scalers[target_name]
    train_predictions_rescaled = scaler_y.inverse_transform(train_predictions)
    test_predictions_rescaled = scaler_y.inverse_transform(test_predictions)

    y_train_rescaled = scaler_y.inverse_transform(y_train[[target_name]])
    y_test_rescaled = scaler_y.inverse_transform(y_test[[target_name]])

    train_r2 = r2_score(y_train_rescaled, train_predictions_rescaled)
    test_r2 = r2_score(y_test_rescaled, test_predictions_rescaled)
    mae = mean_absolute_error(y_test_rescaled, test_predictions_rescaled)
    mse = mean_squared_error(y_test_rescaled, test_predictions_rescaled)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test_rescaled - test_predictions_rescaled) / y_test_rescaled)) * 100

    # Add results to the master results table
    new_results = {
        "Target": target_name,
        "Model": "Hyper-Tuned LSTM",
        "Type": "Neural Network",
        "Train R²": train_r2,
        "Test R²": test_r2,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape,
        "Comments": "Working Well" if test_r2 > 0.75 and mape < 10 else "Needs Improvement"
    }

    # Convert new_results to a DataFrame
    new_results_df = pd.DataFrame([new_results])

    # Check if the target and model already exist in the master results table
    existing_row_index = master_resultsStd[
        (master_resultsStd["Target"] == new_results["Target"]) & (master_resultsStd["Model"] == new_results["Model"])
    ].index

    if len(existing_row_index) > 0:
        # Update the existing row by replacing its values
        master_resultsStd.loc[existing_row_index, :] = new_results_df.iloc[0].values
    else:
        # Append the new results
        master_resultsStd = pd.concat([master_resultsStd, new_results_df], ignore_index=True)

# Save updated results
master_resultsStd.to_csv(master_results_file_path, index=False)
print(f"[INFO] Updated master results saved to '{master_results_file_path}'.")

# Display the final results table
print("\nFinal Master Results Table:")
print(master_resultsStd.to_string(index=False))

[INFO] Loaded existing master results table from '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/master_resultsStd.csv'.
Reloading Tuner from hyperparam_tuning/lstm_hyper_tuning/tuner0.json


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.3721 - val_loss: 0.0169
Epoch 2/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0705 - val_loss: 0.0101
Epoch 3/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0567 - val_loss: 0.0069
Epoch 4/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 0.0533 - val_loss: 0.0089
Epoch 5/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0518 - val_loss: 0.0060
Epoch 6/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 0.0511 - val_loss: 0.0056
Epoch 7/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0514 - val_loss: 0.0092
Epoch 8/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0498 - val_loss: 0.0040
Epoch 9/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0484 - val_loss: 0.0038
Epoch 10/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0487 - val_loss: 0.0042
Epoch 11/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0482 - val_loss: 0.0060
Epoch 12/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 8s 8

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


909/909 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.2372 - val_loss: 0.0080
Epoch 2/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0527 - val_loss: 0.0038
Epoch 3/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.0462 - val_loss: 0.0037
Epoch 4/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0445 - val_loss: 0.0011
Epoch 5/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0440 - val_loss: 0.0020
Epoch 6/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0439 - val_loss: 0.0029
Epoch 7/20
909/909 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0446 - val_loss: 0.0020
1817/1817 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[INFO] Updated master results saved to '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/master_resultsStd.csv'.

Final Master Results Table:
   Target                       Model           Train R²  Test R²          MAE          MSE         RMSE         MAPE     Comments                     Type
new_price       

# BEST HYPER TUNED ARCHITECTURE

In [29]:
# Display the best architecture and hyperparameters from the hyperparameter tuner
print("[INFO] Best Hyperparameters for the Tuned LSTM Model:")

# Number of layers
num_layers = best_hps.get('num_layers')
print(f"Number of Layers: {num_layers}")

# Loop through each layer and display its configuration
for i in range(num_layers):
    units = best_hps.get(f'units_layer_{i+1}')
    dropout = best_hps.get(f'dropout_layer_{i+1}')
    recurrent_dropout = best_hps.get(f'recurrent_dropout_{i+1}')
    return_sequences = best_hps.get(f'return_sequences_{i+1}') if i < (num_layers - 1) else False
    print(f"Layer {i+1}:")
    print(f"  Units: {units}")
    print(f"  Dropout Rate: {dropout}")
    print(f"  Recurrent Dropout Rate: {recurrent_dropout}")
    print(f"  Return Sequences: {return_sequences}")

# Activation function
activation_function = best_hps.get('activation')
print(f"Activation Function: {activation_function}")

# Optimizer
optimizer = best_hps.get('optimizer')
print(f"Optimizer: {optimizer}")

# Loss function
loss_function = best_hps.get('loss')
print(f"Loss Function: {loss_function}")

# Learning rate
learning_rate = best_hps.get('learning_rate')
print(f"Learning Rate: {learning_rate}")

# Display manually tuned batch size and epochs
print("\n[INFO] Manually Tuned Hyperparameters:")
batch_size = 64  # Replace with the batch size used in the nested loop
epochs = 20  # Replace with the epochs used in the nested loop
print(f"Batch Size: {batch_size}")
print(f"Epochs: {epochs}")

# Alternatively, show a summary of the best model architecture
print("\n[INFO] Best Model Architecture:")
best_model.summary()

[INFO] Best Hyperparameters for the Tuned LSTM Model:
Number of Layers: 3
Layer 1:
  Units: 64
  Dropout Rate: 0.2
  Recurrent Dropout Rate: 0.0
  Return Sequences: 1
Layer 2:
  Units: 128
  Dropout Rate: 0.2
  Recurrent Dropout Rate: 0.0
  Return Sequences: 1
Layer 3:
  Units: 128
  Dropout Rate: 0.3
  Recurrent Dropout Rate: 0.0
  Return Sequences: False
Activation Function: relu
Optimizer: adam
Loss Function: mse
Learning Rate: 0.001

[INFO] Manually Tuned Hyperparameters:
Batch Size: 64
Epochs: 20

[INFO] Best Model Architecture:


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_20 (LSTM)                       │ (None, 1, 64)               │          34,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_22 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 794,501 (3.03 MB)

 Trainable params: 264,833 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 529,668 (2.02 MB)

In [30]:
import os
from google.colab import drive

# Install `python-docx` if not already installed
try:
    from docx import Document
except ModuleNotFoundError:
    print("[INFO] Installing python-docx...")
    !pip install python-docx
    from docx import Document

# Mount Google Drive (specific to Google Colab)
try:
    drive.mount('/content/drive', force_remount=True)
    print("[INFO] Google Drive mounted successfully.")
except Exception as e:
    print(f"[WARNING] Could not mount Google Drive: {e}")
    print("[INFO] Ensure this code is running in Google Colab.")

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Step 1: Save the results of the best hyperparameter-tuned architecture
doc = Document()

# Add a title to the document
doc.add_heading("Best Hyperparameter-Tuned Architecture", level=1)

# Add the architecture details to the document
doc.add_heading("Hyperparameters", level=2)

try:
    # Ensure best_hps is defined
    if 'best_hps' not in globals():
        raise ValueError("The variable 'best_hps' is not defined. Ensure hyperparameter tuning is completed.")

    # Add the number of layers
    num_layers = best_hps.get('num_layers')
    doc.add_paragraph(f"Number of Layers: {num_layers}")

    # Add details for each layer
    for i in range(num_layers):
        units = best_hps.get(f'units_layer_{i+1}')
        dropout = best_hps.get(f'dropout_layer_{i+1}')
        recurrent_dropout = best_hps.get(f'recurrent_dropout_{i+1}')
        return_sequences = best_hps.get(f'return_sequences_{i+1}') if i < (num_layers - 1) else False
        doc.add_paragraph(f"Layer {i+1}:")
        doc.add_paragraph(f"  Units: {units}")
        doc.add_paragraph(f"  Dropout Rate: {dropout}")
        doc.add_paragraph(f"  Recurrent Dropout Rate: {recurrent_dropout}")
        doc.add_paragraph(f"  Return Sequences: {return_sequences}")

    # Add other hyperparameters
    activation_function = best_hps.get('activation')
    optimizer = best_hps.get('optimizer')
    loss_function = best_hps.get('loss')
    learning_rate = best_hps.get('learning_rate')

    doc.add_paragraph(f"Activation Function: {activation_function}")
    doc.add_paragraph(f"Optimizer: {optimizer}")
    doc.add_paragraph(f"Loss Function: {loss_function}")
    doc.add_paragraph(f"Learning Rate: {learning_rate}")

    # Add batch size and epochs (if manually tuned)
    batch_size = 64  # Replace with the actual batch size used
    epochs = 20  # Replace with the actual epochs used
    doc.add_paragraph(f"Batch Size: {batch_size}")
    doc.add_paragraph(f"Epochs: {epochs}")

except Exception as e:
    print(f"[ERROR] An error occurred while adding hyperparameters to the Word document: {e}")
    raise

# Add a placeholder for the model summary
try:
    if 'best_model' not in globals():
        raise ValueError("The variable 'best_model' is not defined. Ensure the best model is built after tuning.")

    doc.add_heading("Model Architecture Summary", level=2)

    # Capture the model summary as a string
    from io import StringIO
    import sys
    model_summary_io = StringIO()
    sys.stdout = model_summary_io  # Redirect stdout to capture the model summary
    best_model.summary()
    sys.stdout = sys.__stdout__  # Reset stdout to default
    model_summary = model_summary_io.getvalue()

    # Add the model summary to the Word document
    doc.add_paragraph(model_summary)

except Exception as e:
    print(f"[ERROR] An error occurred while adding the model summary to the Word document: {e}")
    raise

# Step 4: Save the Word document
output_file = os.path.join(subdirectory, 'best_hyper_tuned_architecture.docx')

try:
    doc.save(output_file)
    print(f"[INFO] Word document saved successfully at '{output_file}'.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the Word document: {e}")
    raise

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_20 (LSTM)                       │ (None, 1, 64)               │          34,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_22 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 794,501 (3.03 MB)

 Trainable params: 264,833 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 529,668 (2.02 MB)

Mounted at /content/drive
[INFO] Google Drive mounted successfully.


In [26]:
import os
from google.colab import drive

# Install `python-docx` if not already installed
try:
    from docx import Document
except ModuleNotFoundError:
    print("[INFO] Installing python-docx...")
    !pip install python-docx
    from docx import Document

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Step 1: Save the results of the best hyperparameter-tuned architecture
doc = Document()

# Add a title to the document
doc.add_heading("Best Hyperparameter-Tuned Architecture", level=1)

# Add the architecture details to the document
doc.add_heading("Hyperparameters", level=2)
doc.add_paragraph(f"Number of Layers: {best_hps.get('num_layers')}")

for i in range(best_hps.get('num_layers')):
    units = best_hps.get(f'units_layer_{i+1}')
    dropout = best_hps.get(f'dropout_layer_{i+1}')
    doc.add_paragraph(f"Layer {i+1}: {units} units, Dropout rate: {dropout}")

learning_rate = best_hps.get('learning_rate')
doc.add_paragraph(f"Learning Rate: {learning_rate}")

# Add a placeholder for the model summary
doc.add_heading("Model Architecture Summary", level=2)
doc.add_paragraph("[INFO] Model Architecture Summary:")
doc.add_paragraph(best_model.summary(print_fn=lambda x: x))

# Step 4: Save the Word document
output_file = os.path.join(subdirectory, 'best_hyper_tuned_architecture.docx')

try:
    doc.save(output_file)
    print(f"[INFO] Word document saved successfully at '{output_file}'.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the Word document: {e}")
    raise

Mounted at /content/drive


[INFO] Word document saved successfully at '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/best_hyper_tuned_architecture.docx'.


# SAVING THE RESULTS

In [31]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Define file path for saving the DataFrame as 'Std_Datasetv6.csv'
file_path = os.path.join(subdirectory, 'Std_Datasetv6.csv')

# Save the DataFrame to the file path
try:
    merged_dataset.to_csv(file_path, index=False)
    print(f"[INFO] File 'Std_Datasetv6.csv' saved successfully at '{file_path}'.")
except NameError:
    print("[ERROR] The DataFrame 'merged_dataset' does not exist. Please define it before saving.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the file: {e}")

In [32]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Define file path for saving the DataFrame
file_path = os.path.join(subdirectory, 'master_resultsStd.csv')  # Change the filename

# Save the DataFrame to the file path
try:
    master_resultsStd.to_csv(file_path, index=False)
    print(f"[INFO] File 'master_resultsStd.csv' saved successfully at '{file_path}'.")
except NameError:
    print("[ERROR] The DataFrame 'master_resultsStd' does not exist. Please define it before saving.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the file: {e}")

In [33]:
import os
from google.colab import drive

# Install `python-docx` if not already installed
try:
    from docx import Document
except ModuleNotFoundError:
    print("[INFO] Installing python-docx...")
    !pip install python-docx
    from docx import Document

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Step 1: Use the existing DataFrame
try:
    df = master_resultsStd.head(20)  # Use the first 20 rows of the already loaded DataFrame
    print("[INFO] DataFrame 'master_resultsStd' is already loaded and limited to the first 20 rows.")
except NameError:
    print("[ERROR] The DataFrame 'master_resultsStd' does not exist. Please ensure it is loaded.")
    raise
except Exception as e:
    print(f"[ERROR] An unexpected error occurred: {e}")
    raise

# Step 2: Create a Word document
doc = Document()

# Add a title to the document
doc.add_heading("Master Results Table", level=1)

# Step 3: Add a table to the Word document
table = doc.add_table(rows=1, cols=len(df.columns))

# Add column headers to the table
header_cells = table.rows[0].cells
for i, column_name in enumerate(df.columns):
    header_cells[i].text = column_name

# Add data rows to the table
for _, row in df.iterrows():
    row_cells = table.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Step 4: Save the Word document to the subdirectory
output_file = os.path.join(subdirectory, 'master_resultsStd.doc')  # Save as master_resultsStd.doc

try:
    doc.save(output_file)
    print(f"[INFO] Word document saved successfully at '{output_file}'.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the Word document: {e}")
    raise

In [37]:
master_resultsStd.shape

(14, 10)

In [38]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Ensure the subdirectory exists
os.makedirs(subdirectory, exist_ok=True)

# Define file path for saving the DataFrame
file_path = os.path.join(subdirectory, 'master_resultsStd.csv')  # Change the filename

# Save the DataFrame to the file path
try:
    master_resultsStd.to_csv(file_path, index=False)
    print(f"[INFO] File 'master_resultsStd.csv' saved successfully at '{file_path}'.")
except NameError:
    print("[ERROR] The DataFrame 'master_resultsStd' does not exist. Please define it before saving.")
except Exception as e:
    print(f"[ERROR] An error occurred while saving the file: {e}")

In [39]:
merged_dataset.shape

(72663, 71)

In [40]:
master_resultsStd.head(20)

,Target,Model,Train R²,Test R²,MAE,MSE,RMSE,MAPE,Comments,Type
0,new_price,Linear Regression,0.6297378908089446,0.627819,4.893183e-01,3.739195e-01,6.114896e-01,2.493082e+02,Normal,Simple Linear Regression
1,new_price,Random Forest Regressor,0.973289789332776,0.824091,2.823479e-01,1.767307e-01,4.203935e-01,1.389046e+02,Normal,Decision Tree Model
2,new_price,XGBoost Regressor,0.8270074320236708,0.766067,3.657201e-01,2.350260e-01,4.847948e-01,1.733921e+02,Normal,Decision Tree Model
3,new_price,Gradient Boosting Regressor,0.7699284396872543,0.746053,3.839207e-01,2.551336e-01,5.051075e-01,1.949614e+02,Normal,Decision Tree Model
4,PC,Linear Regression,0.973660068974925,0.974082,1.317771e-01,2.591456e-02,1.609800e-01,2.944177e+01,Normal,Simple Linear Regression
5,PC,Random Forest Regressor,1.0,1.000000,1.665155e-14,4.991350e-28,2.234133e-14,1.707175e-12,Normal,Decision Tree Model
6,PC,XGBoost Regressor,0.9999566580170176,0.999957,6.098938e-03,4.334612e-05,6.583777e-03,6.560287e-01,Normal,Decision Tree Model
7,PC,Gradient Boosting Regressor,0.9999999999999998,1.000000,1.221103e-08,2.160188e-16,1.469758e-08,3.735057e-06,Normal,Decision Tree Model
8,new_price,Base LSTM,0.995441277667468,0.995389,1.088874e+05,2.124968e+10,1.457727e+05,1.696912e+00,Working Well,Neural Network
9,PC,Base LSTM,0.9986385895087232,0.998622,6.581006e-01,6.840771e-01,8.270895e-01,1.799154e-01,Working Well,Neural Network
